In [7]:
%%file statistics.py 
from typing import Sequence, Tuple, Union

def iqr(values: Sequence[float]) -> float:

    if len(values) <= 1: return 0.

    values = sorted(values)

    p25,p75 = percentile(values, [0.25,0.75])

    return p75-p25

def percentile(values: Sequence[float], percentiles: Union[float,Sequence[float]]) -> Union[float, Tuple[float,...]]:

    def _percentile(values: Sequence[float], percentile: float) -> float:
        assert 0 <= percentile and percentile <= 1, "Percentile must be between 0 and 1 inclusive."

        i = percentile*(len(values)-1)

        if i == int(i):
            return values[int(i)]
        else:
            return values[int(i)] * (1-(i-int(i))) + values[int(i)+1] * (i-int(i))

    values = sorted(values)

    if isinstance(percentiles,(float,int)):
        return _percentile(values, percentiles)
    else:
        return tuple([_percentile(values, p) for p in percentiles ])

class OnlineVariance():
    """Calculate sample variance in an online fashion.
    Remarks:
        This algorithm is known as Welford's algorithm and the implementation below
        is a modified version of the Python algorithm by Wikepedia contirubtors (2020).
    References:
        Wikipedia contributors. (2020, July 6). Algorithms for calculating variance. In Wikipedia, The
        Free Encyclopedia. Retrieved 18:00, July 24, 2020, from
        https://en.wikipedia.org/w/index.php?title=Algorithms_for_calculating_variance&oldid=966329915
    """

    def __init__(self) -> None:
        """Instatiate an OnlineVariance calcualator."""
        self._count    = 0.
        self._mean     = 0.
        self._M2       = 0.
        self._variance = float("nan")

    @property
    def variance(self) -> float:
        """The variance of all given updates."""
        return self._variance

    def update(self, value: float) -> None:
        """Update the current variance with the given value."""

        (count,mean,M2) = (self._count, self._mean, self._M2)

        count   += 1
        delta   = value - mean
        mean   += delta / count
        delta2  = value - mean
        M2     += delta * delta2

        (self._count, self._mean, self._M2) = (count, mean, M2)

        if count > 1:
            self._variance = M2 / (count - 1)

class OnlineMean():
    """Calculate mean in an online fashion."""

    def __init__(self):
        self._n = 0
        self._mean = float('nan')

    @property
    def mean(self) -> float:
        """The mean of all given updates."""

        return self._mean

    def update(self, value:float) -> None:
        """Update the current mean with the given value."""

        self._n += 1

        alpha = 1/self._n

        self._mean = value if alpha == 1 else (1 - alpha) * self._mean + alpha * value

"""test_sets = [ [0, 2], [1, 1], [1,2], [-1,1], [10.5,20] ]
for test_set in test_sets:
    online = OnlineMean()
    for number in test_set:
        online.update(number)"""

Overwriting statistics.py


In [8]:
%%file utils.py
from collections import defaultdict

class HashableDict(dict):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self._hash = hash(tuple(self.items()))

    def __hash__(self) -> int:
        assert self._hash == hash(tuple(self.items()))
        return self._hash

class KeyDefaultDict(defaultdict):
    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError( key )
        else:
            value = self.default_factory(key)
            self[key] = value
            return value

Writing utils.py


In [5]:
online.mean

15.25

In [9]:
import importlib.util

from coba.exceptions import CobaException
from coba.learners import LinUCBLearner

In [12]:
%%time
learner = LinUCBLearner()
probs   = learner.predict(None, [1,1]) #[1,2,3]
probs

CPU times: user 445 µs, sys: 39 µs, total: 484 µs
Wall time: 470 µs


[0.5, 0.5]

In [5]:
probs

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [18]:
learner.learn(None, 1, 1, 1.5, None)

In [21]:
probs   = learner.predict(None, [1,2]) #[1,2,3]
probs

[0.0, 1.0]

In [19]:
learner._A_inv

array([[ 0.57142857, -0.42857143],
       [-0.42857143,  0.57142857]])

In [22]:
learner.params

{'family': 'LinUCB', 'alpha': 1, 'features': [1, 'a', 'ax']}

In [25]:
import numpy as np
actions = ["E04","A01","S01","S02"]
context = [1,2,3]
features = np.array([context for action in actions]).T
features

array([[1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]])

In [26]:
np.zeros(features.shape[0])

array([0., 0., 0.])

In [36]:
v = np.zeros(5)
w=np.identity(5)
((2-1.0)/(1+v)) @ w

array([1., 1., 1., 1., 1.])

In [28]:
v

array([0., 0., 0., 0., 0.])

In [35]:
((2-1.0)/(1+v)).T.shape

(5,)

In [42]:
x1 = np.array([[0.05882353, 0.05882353, 0.05882353, 0.05882353, 0.05882353],
               [0.05882353, 0.05882353, 0.05882353, 0.05882353, 0.05882353],
       [0.05882353, 0.05882353, 0.05882353, 0.05882353, 0.05882353],
       [0.11764706, 0.11764706, 0.11764706, 0.11764706, 0.11764706],
       [0.17647059, 0.17647059, 0.17647059, 0.17647059, 0.17647059]])
                                                                           

x1*x1

array([[0.00346021, 0.00346021, 0.00346021, 0.00346021, 0.00346021],
       [0.00346021, 0.00346021, 0.00346021, 0.00346021, 0.00346021],
       [0.00346021, 0.00346021, 0.00346021, 0.00346021, 0.00346021],
       [0.01384083, 0.01384083, 0.01384083, 0.01384083, 0.01384083],
       [0.03114187, 0.03114187, 0.03114187, 0.03114187, 0.03114187]])

In [43]:
x1@x1

array([[0.02768166, 0.02768166, 0.02768166, 0.02768166, 0.02768166],
       [0.02768166, 0.02768166, 0.02768166, 0.02768166, 0.02768166],
       [0.02768166, 0.02768166, 0.02768166, 0.02768166, 0.02768166],
       [0.05536332, 0.05536332, 0.05536332, 0.05536332, 0.05536332],
       [0.08304498, 0.08304498, 0.08304498, 0.08304498, 0.08304498]])

In [49]:
ll=np.array([[0.05882353]])
ol = np.array([[1]
               [1]
               [1]
               [2]
               [3]])

ll*ol

IndexError: list index out of range